In [5]:
import sqlite3
import csv

# Connect to the database with timeout
db_path = "stat-term.db"
conn = sqlite3.connect(db_path, timeout=30)
cursor = conn.cursor()

# Optimize SQLite settings for import
cursor.execute("PRAGMA journal_mode=WAL")
cursor.execute("PRAGMA synchronous=OFF")

csv_path = "../data/nba_player_stats.csv"

try:
    # Preload players and teams into memory
    players = {}
    teams = {}
    stats_rows = []

    with open(csv_path, newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader, start=1):
            player_name = row["player"]
            team_name = row["team"]

            # Add player if not seen
            if player_name not in players:
                cursor.execute(
                    "INSERT OR IGNORE INTO nba_players (player_name) VALUES (?)",
                    (player_name,),
                )
                cursor.execute(
                    "SELECT player_id FROM nba_players WHERE player_name = ?",
                    (player_name,),
                )
                players[player_name] = cursor.fetchone()[0]

            # Add team if not seen (using team_name column)
            if team_name not in teams:
                cursor.execute(
                    "INSERT OR IGNORE INTO nba_teams (team_name) VALUES (?)",
                    (team_name,),
                )
                cursor.execute(
                    "SELECT team_id FROM nba_teams WHERE team_name = ?", (team_name,)
                )
                teams[team_name] = cursor.fetchone()[0]

            # Collect stats row with float-to-int conversion
            stats_rows.append(
                (
                    players[player_name],
                    teams[team_name],
                    row["season"],
                    int(float(row["rk"])) if row["rk"] else None,  # Handle decimals
                    player_name,
                    int(float(row["age"])) if row["age"] else None,
                    team_name,
                    row["pos"],
                    int(float(row["g"])) if row["g"] else None,
                    int(float(row["gs"])) if row["gs"] else None,
                    float(row["mp"]) if row["mp"] else None,
                    int(float(row["fg"])) if row["fg"] else None,
                    int(float(row["fga"])) if row["fga"] else None,
                    float(row["fg%"]) if row["fg%"] else None,
                    int(float(row["3p"])) if row["3p"] else None,
                    int(float(row["3pa"])) if row["3pa"] else None,
                    float(row["3p%"]) if row["3p%"] else None,
                    int(float(row["2p"])) if row["2p"] else None,
                    int(float(row["2pa"])) if row["2pa"] else None,
                    float(row["2p%"]) if row["2p%"] else None,
                    float(row["efg%"]) if row["efg%"] else None,
                    float(row["ft"]) if row["ft"] else None,
                    int(float(row["fta"])) if row["fta"] else None,
                    float(row["ft%"]) if row["ft%"] else None,
                    int(float(row["orb"])) if row["orb"] else None,
                    int(float(row["drb"])) if row["drb"] else None,
                    int(float(row["trb"])) if row["trb"] else None,
                    int(float(row["ast"])) if row["ast"] else None,
                    int(float(row["stl"])) if row["stl"] else None,
                    int(float(row["blk"])) if row["blk"] else None,
                    int(float(row["tov"])) if row["tov"] else None,
                    int(float(row["pf"])) if row["pf"] else None,
                    int(float(row["pts"])) if row["pts"] else None,
                    row["awards"] if row["awards"] else None,
                )
            )

            # Print progress for each row
            print(f"Processed row {i}: {player_name}, {row['season']}")

    # Batch insert stats
    cursor.executemany(
        """
        INSERT OR REPLACE INTO nba_season_player_stats (
            player_id, team_id, season, rk, player_name, age, team, pos, g, gs, mp, 
            fg, fga, fg_pct, three_p, three_pa, three_p_pct, two_p, two_pa, two_p_pct, 
            efg_pct, ft, fta, ft_pct, orb, drb, trb, ast, stl, blk, tov, pf, pts, awards
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
        stats_rows,
    )

    conn.commit()
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    cursor.execute("PRAGMA synchronous=FULL")
    conn.close()

print("CSV data imported successfully!")

Processed row 1: Joel Embiid, 2023-2024
Processed row 2: Luka Doncic, 2023-2024
Processed row 3: Giannis Antetokounmpo, 2023-2024
Processed row 4: Shai Gilgeous-Alexander, 2023-2024
Processed row 5: Jalen Brunson, 2023-2024
Processed row 6: Devin Booker, 2023-2024
Processed row 7: Kevin Durant, 2023-2024
Processed row 8: Jayson Tatum, 2023-2024
Processed row 9: De'Aaron Fox, 2023-2024
Processed row 10: Donovan Mitchell, 2023-2024
Processed row 11: Stephen Curry, 2023-2024
Processed row 12: Nikola Jokic, 2023-2024
Processed row 13: Anthony Edwards, 2023-2024
Processed row 14: Tyrese Maxey, 2023-2024
Processed row 15: Lebron James, 2023-2024
Processed row 16: Trae Young, 2023-2024
Processed row 17: Kyrie Irving, 2023-2024
Processed row 18: Ja Morant, 2023-2024
Processed row 19: Anthony Davis, 2023-2024
Processed row 20: Damian Lillard, 2023-2024
Processed row 21: Demar Derozan, 2023-2024
Processed row 22: Julius Randle, 2023-2024
Processed row 23: Lamelo Ball, 2023-2024
Processed row 24: